# Using Biopython's PDB module to list resolved residues and construct fit commands

Most structure models have at least a few atoms/residues that cannot be resolved even if most of the molecule's chain is otherwise observed. These will cause gaps in the specific atoms represented in the final model. And of course, different models can have different missing atoms.  The upshot is that even for the same chain, you often cannot simply specify that a  fit / docked/ structurally align command act on the same chain in a different structure.  In particular for Pymol, you need to know matching atoms to get Pymol's pair_fit commands to work. With complex structures with several chains, it can be problematic to do the determination of matching atoms by hand. This notebook shows how to determine what is there and builds up in steps to generating the pair_fit commands for the same chains shared by different structures.

The fit commands are intended to be used in Pymol.

Think of this as complementing my notebook about 'missing' residues. Depending on what you are trying to do (or use as the source of information), you may want this notebook or that one.

In [1]:
#get stucture
!curl -OL https://files.rcsb.org/download/6AGB.pdb.gz
!gunzip 6AGB.pdb.gz
!curl -OL https://files.rcsb.org/download/6AH3.pdb.gz
!gunzip 6AH3.pdb.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  491k  100  491k    0     0  1118k      0 --:--:-- --:--:-- --:--:-- 1118k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  519k  100  519k    0     0  1269k      0 --:--:-- --:--:-- --:--:-- 1266k


In [2]:
# Get a list of resolved residue positions for making into ranges that can be used in molecular visualization commands
# relies on use of `not residue.id[0].startswith('H_')` to exclude heteroatoms. (see under 'What is a residue id?' 
# at https://biopython.org/wiki/The_Biopython_Structural_Bioinformatics_FAQ).
# For example, without use of `not residue.id[0].startswith('H_')` to exclude heteroatoms
# here a heteroatom zinc would be included as part of chain K and numbered as 201 and 
# that would cause issues with the range numbering listing a lone residue that isn't
# actually part of chain K.
from Bio.PDB import *
from collections import defaultdict
structure = PDBParser().get_structure('6AGB', '6AGB.pdb')
residues_resolved_per_chain = defaultdict(list)
for model in structure:
    for chain in model:
        for residue in chain:
            #print (str(chain.id),residue.id[1])
            #print (str(chain.id),residue.id)
            '''
            if str(chain.id) == 'K':
                print (residue.id[0])
                '''
            #print [residue.id[0]]
            if not residue.id[0].startswith('H_'):
                residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
print ('List of resolved residues from first chain:')
print (residues_resolved_per_chain[list(residues_resolved_per_chain.keys())[0]]) #to get first with Python 3 without iter
#print ('chains:',residues_resolved_per_chain.keys()) 
# SUMMARIZE RESOLVED RESIDUES
print ("\nTotal resolved per chain:")
for chain in residues_resolved_per_chain:
    print (chain, len(residues_resolved_per_chain[chain]))

List of resolved residues from first chain:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213

In [3]:
# Get ranges of resolved residue positions that can be used in molecular visualization commands
from Bio.PDB import *
from collections import defaultdict

def range_extract(lst):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    'Yield 2-tuple ranges or 1-tuple single elements from list of increasing ints'
    lenlst = len(lst)
    i = 0
    while i< lenlst:
        low = lst[i]
        while i <lenlst-1 and lst[i]+1 == lst[i+1]: i +=1
        hi = lst[i]
        if   hi - low >= 2:
            yield (low, hi)
        elif hi - low == 1:
            yield (low,)
            yield (hi,)
        else:
            yield (low,)
        i += 1
def printr(ranges):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    print( ','.join( (('%i-%i' % r) if len(r) == 2 else '%i' % r)
                     for r in ranges ) )

    
def get_nice_ranges(ranges):
    return [ (('%i-%i' % r) if len(r) == 2 else '%i' % r) for r in ranges ]
    
structure = PDBParser().get_structure('6AGB', '6AGB.pdb')
residues_resolved_per_chain = defaultdict(list)
for model in structure:
    for chain in model:
        for residue in chain:
            #print (str(chain.id),residue.id[1])
            #print (str(chain.id),residue.id)
            '''
            if str(chain.id) == 'K':
                print (residue.id[0])
                '''
            #print [residue.id[0]]
            if not residue.id[0].startswith('H_'):
                residues_resolved_per_chain[str(chain.id)].append(residue.id[1])

#print (residues_resolved_per_chain[list(residues_resolved_per_chain.keys())[0]]) #to get first with Python 3 without iter
#print ('chains:',residues_resolved_per_chain.keys()) 

print ("Residues resolved per chain, concisely stated:")
ranges_o_residues_resolved_per_chain = {}
for k,lst in residues_resolved_per_chain.items():
    print(k)
    #print (list(range_extract(lst))) # to see the list of tuples unformatted
    printr(range_extract(lst)) # tuples formatted as ranges
    ranges_o_residues_resolved_per_chain[k] = get_nice_ranges(range_extract(lst))
ranges_o_residues_resolved_per_chain

Residues resolved per chain, concisely stated:
A
1-369
B
54-124,139-524,530-685,696-742,752-875
C
14-188
D
1-24,77-279
E
2-147
F
2-158
G
13-107,115-140
H
3-133
I
1-242
J
1-293
K
17-144


{'A': ['1-369'],
 'B': ['54-124', '139-524', '530-685', '696-742', '752-875'],
 'C': ['14-188'],
 'D': ['1-24', '77-279'],
 'E': ['2-147'],
 'F': ['2-158'],
 'G': ['13-107', '115-140'],
 'H': ['3-133'],
 'I': ['1-242'],
 'J': ['1-293'],
 'K': ['17-144']}

## Towards writing commands for fitting matching residues of shared chains
: extract positions of resolved residues shared between two structures

In [4]:
#get stucture
!curl -OL https://files.rcsb.org/download/3IAB.pdb.gz
!gunzip 3IAB.pdb.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70953  100 70953    0     0   256k      0 --:--:-- --:--:-- --:--:--  256k


In [5]:
# positions of resolved residues shared between two structures
# Note both chains in 3iab include selenomethionine (MSE) that are listed as heteroatoms. However, they have alpha-carbons 
# & should be used in the fit. Thus I need to change the handling of `not residue.id[0].startswith('H_')` to accomodate
# selenomehtionine. (Found because without adjusting, I was seeing breaks in residues mathcing up where
# I had not noticed before when I did by hand.)
allow_MSE = True

from Bio.PDB import *
from collections import defaultdict

def range_extract(lst):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    'Yield 2-tuple ranges or 1-tuple single elements from list of increasing ints'
    lenlst = len(lst)
    i = 0
    while i< lenlst:
        low = lst[i]
        while i <lenlst-1 and lst[i]+1 == lst[i+1]: i +=1
        hi = lst[i]
        if   hi - low >= 2:
            yield (low, hi)
        elif hi - low == 1:
            yield (low,)
            yield (hi,)
        else:
            yield (low,)
        i += 1
def printr(ranges):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    print( ','.join( (('%i-%i' % r) if len(r) == 2 else '%i' % r)
                     for r in ranges ) )

    
def get_nice_ranges(ranges):
    return [ (('%i-%i' % r) if len(r) == 2 else '%i' % r) for r in ranges ]

# Read in structure a ('first' structure)
structure = PDBParser().get_structure('6AGB', '6AGB.pdb')
residues_resolved_per_chain = defaultdict(list)
for model in structure:
    for chain in model:
        for residue in chain:
            #print (str(chain.id),residue.id[1])
            #print (str(chain.id),residue.id)
            '''
            if str(chain.id) == 'K':
                print (residue.id[0])
                '''
            #print [residue.id[0]]
            if allow_MSE:
                if residue.id[0] == 'H_MSE':
                    residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
                else:
                    if not residue.id[0].startswith('H_'):
                        residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
            else:
                if not residue.id[0].startswith('H_'):
                    residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
# Read in structure a                
structure_b = PDBParser().get_structure('3IAB', '3IAB.pdb')
residues_resolved_per_chain_b = defaultdict(list)
for model_b in structure_b:
    for chain_b in model_b:
        for residue_b in chain_b:
            #print (str(chain_b.id),residue_b.id[1])
            #print (str(chain_b.id),residue_b.id)
            '''
            if str(chain_b.id) == 'K':
                print (residue_b.id[0])
                '''
            #print [residue_b.id[0]]
            if allow_MSE:
                if residue_b.id[0] == 'H_MSE':
                    residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])
                else:
                    if not residue_b.id[0].startswith('H_'):
                        residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])
            else:
                if not residue_b.id[0].startswith('H_'):
                    residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])

chain_pairs = [('F','A'),('G','B')] #putting chain identiifier from 'first' structure first in each pair.
                
#now that have position info for residues of both structures, get the shared positions
shared_positions_per_chain_pairs = {}
for chain_pair in chain_pairs:
    shared = list(set(residues_resolved_per_chain[chain_pair[0]]) & set(residues_resolved_per_chain_b[chain_pair[1]]) )
    shared_positions_per_chain_pairs[chain_pair] = shared

#Now format more concisely
print ("Shared positions, concisely stated:")
ranges_o_residues_resolved_per_chain_pairs = {}
for k,lst in shared_positions_per_chain_pairs.items():
    print(k)
    #print (list(range_extract(lst))) # to see the list of tuples unformatted
    printr(range_extract(lst)) # tuples formatted as ranges
    ranges_o_residues_resolved_per_chain_pairs[k] = get_nice_ranges(range_extract(lst))

# residues_resolved_per_chain_b
shared_positions_per_chain_pairs
ranges_o_residues_resolved_per_chain_pairs

Shared positions, concisely stated:
('F', 'A')
4-121,129-158
('G', 'B')
14-104,125-140


/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3590.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3593.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3609.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain R is discontinuous at line 3626.
  PDBConstructionWarning)


{('F', 'A'): ['4-121', '129-158'], ('G', 'B'): ['14-104', '125-140']}

## Writing commands for fitting matching residues of chains shared between two structures

The output can be used in Pymol to structurally align the chains.

In [6]:
# positions of resolved residues shared between two structures formatted into pair_fit commands
allow_MSE = True

from Bio.PDB import *
from collections import defaultdict

def range_extract(lst):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    'Yield 2-tuple ranges or 1-tuple single elements from list of increasing ints'
    lenlst = len(lst)
    i = 0
    while i< lenlst:
        low = lst[i]
        while i <lenlst-1 and lst[i]+1 == lst[i+1]: i +=1
        hi = lst[i]
        if   hi - low >= 2:
            yield (low, hi)
        elif hi - low == 1:
            yield (low,)
            yield (hi,)
        else:
            yield (low,)
        i += 1
def printr(ranges):
    # from https://www.rosettacode.org/wiki/Range_extraction#Python
    print( ','.join( (('%i-%i' % r) if len(r) == 2 else '%i' % r)
                     for r in ranges ) )

    
def get_nice_ranges(ranges):
    return [ (('%i-%i' % r) if len(r) == 2 else '%i' % r) for r in ranges ]

def get_nice_ranges_wcolon(ranges):
    return [ (('%i:%i' % r) if len(r) == 2 else '%i' % r) for r in ranges ]

# Read in structure a ('first' structure)
structure = PDBParser().get_structure('3IAB', '3IAB.pdb')
residues_resolved_per_chain = defaultdict(list)
for model in structure:
    for chain in model:
        for residue in chain:
            #print (str(chain.id),residue.id[1])
            #print (str(chain.id),residue.id)
            '''
            if str(chain.id) == 'K':
                print (residue.id[0])
                '''
            #print [residue.id[0]]
            if allow_MSE:
                if residue.id[0] == 'H_MSE':
                    residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
                else:
                    if not residue.id[0].startswith('H_'):
                        residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
            else:
                if not residue.id[0].startswith('H_'):
                    residues_resolved_per_chain[str(chain.id)].append(residue.id[1])
# Read in structure a                
structure_b = PDBParser().get_structure('6AGB', '6AGB.pdb')
residues_resolved_per_chain_b = defaultdict(list)
for model_b in structure_b:
    for chain_b in model_b:
        for residue_b in chain_b:
            #print (str(chain_b.id),residue_b.id[1])
            #print (str(chain_b.id),residue_b.id)
            '''
            if str(chain_b.id) == 'K':
                print (residue_b.id[0])
                '''
            #print [residue_b.id[0]]
            if allow_MSE:
                if residue_b.id[0] == 'H_MSE':
                    residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])
                else:
                    if not residue_b.id[0].startswith('H_'):
                        residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])
            else:
                if not residue_b.id[0].startswith('H_'):
                    residues_resolved_per_chain_b[str(chain_b.id)].append(residue_b.id[1])

# further prep for last section
structures = (structure,structure_b) #will need for calling each in the order used later, i.e. structure a and b

chain_pairs = [('A','F'),('B','G')] #putting chain identiifier from 'first' structure first in each pair.
                
#now that have position info for residues of both structures, get the shared positions
shared_positions_per_chain_pairs = {}
for chain_pair in chain_pairs:
    shared = list(set(residues_resolved_per_chain[chain_pair[0]]) & set(residues_resolved_per_chain_b[chain_pair[1]]) )
    shared_positions_per_chain_pairs[chain_pair] = shared

#Now format more concisely
print ("Shared positions, concisely stated:")
ranges_o_residues_resolved_per_chain_pairs = {}
for k,lst in shared_positions_per_chain_pairs.items():
    print(k)
    #print (list(range_extract(lst))) # to see the list of tuples unformatted
    printr(range_extract(lst)) # tuples formatted as ranges
    ranges_o_residues_resolved_per_chain_pairs[k] = get_nice_ranges_wcolon(range_extract(lst))
# form commands
formatted_commands = ""
for chain_pair in ranges_o_residues_resolved_per_chain_pairs:
    for indx,chain in enumerate(chain_pair):
        formatted_commands += f"select {structures[indx].id}ch{chain}CA," #`CA` at end stands for `name CA` / alpha-carbon
        formatted_commands += "|".join( f" {structures[indx].id} and resid {pos_range} and chain {chain} and name CA " 
                                       for pos_range in ranges_o_residues_resolved_per_chain_pairs[chain_pair] ) 
        #for pos_range in ranges_o_residues_resolved_per_chain_pairs[chain_pair]:
         #   formatted_commands += f"{structures[indx].id} and resid {pos_range} and chain {chain} and name CA"
        formatted_commands += "\n"

# residues_resolved_per_chain_b
shared_positions_per_chain_pairs
ranges_o_residues_resolved_per_chain_pairs
print("\n\n")
print ("FORMATTED PYMOL COMMANDS:")
print(" ")
print(formatted_commands)

/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3590.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3593.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3609.
  PDBConstructionWarning)
/srv/conda/lib/python3.6/site-packages/Bio/PDB/StructureBuilder.py:90: PDBConstructionWarning: WARNING: Chain R is discontinuous at line 3626.
  PDBConstructionWarning)


Shared positions, concisely stated:
('A', 'F')
4-121,129-158
('B', 'G')
14-104,125-140



FORMATTED PYMOL COMMANDS:
 
select 3IABchACA, 3IAB and resid 4:121 and chain A and name CA | 3IAB and resid 129:158 and chain A and name CA 
select 6AGBchFCA, 6AGB and resid 4:121 and chain F and name CA | 6AGB and resid 129:158 and chain F and name CA 
select 3IABchBCA, 3IAB and resid 14:104 and chain B and name CA | 3IAB and resid 125:140 and chain B and name CA 
select 6AGBchGCA, 6AGB and resid 14:104 and chain G and name CA | 6AGB and resid 125:140 and chain G and name CA 



-----